# Fundamentals of Genetic Algorithms in DEAP

Welcome to the first code walkthrough for EVAC. Here we will be taking a look at the elements that comprise a basic evolutionary algorithm using the package DEAP in Python.

# Elements to take into account using any evolutionary algorithms

* **Individual representation**
* **Evaluation** and **fitness assignment**

The above two are key considerations that need the most thought. But there's also:

* **Selection**
* **Variation**, produced by applying operators, such as **crossover**, and **mutation**
* **Stopping criterion**, that determines when the algorithm shoulod be stopped, either because the optimum was reach or because the optimization process is not progressing.

## The general structure of a generic algorithm

    def evolutionary_algorithm():

        population = [] # a list with all the individuals in the population

        population =  initialize_population(pop_size)
        t = 0

        while not stop_criterion( population[t] ):
            fitnesses = evaluate( population[t] )
            populations[t+1] = environmental_selection( population[t], offspring )
            offspring = mating_and_mutation( population[t], fitnesses )
            t = t+1

## DEAP: A Python library for evolutionary computation

https://deap.readthedocs.io/en/master/




# Essential features of a DEAP Genetic Algorithm

- deap.creator: meta-factory allowing to create classes that will fulfill the needs of your evolutionary algorithms.
- deap.base.Toolbox: A toolbox for evolution that contains the evolutionary operators. You may populate the toolbox with any other function by using the register() method
- deap.base.Fitness([values]): The fitness is a measure of quality of a solution. If values are provided as a tuple, the fitness is initalized using those values, otherwise it is empty (or invalid). You should inherit from this class to define your custom fitnesses.

# Defining an individual and their representation

First import the required modules and register the different functions required to create individuals that are a list of floats with a minimizin objective
fitness.

In [ ]:
import random

from deap import base
from deap import creator
from deap import tools

In [ ]:
IND_SIZE = 5

In [ ]:
creator.create("FitnessMin", base.Fitness, weights=(-1.0,))
creator.create("Individual", list, fitness=creator.FitnessMin)

toolbox1 = base.Toolbox()
toolbox1.register("attr_float", random.random)
toolbox1.register("individual", tools.initRepeat, creator.Individual,
                 toolbox1.attr_float, n=IND_SIZE)

The first individual can now be constructed

In [ ]:
ind1 = toolbox1.individual()

Printing the individual ind1 and checking if its fitness is valid will give something like this

In [ ]:
print(ind1)

In [ ]:
ind1.fitness.valid

The individual is printed as its base class representation (here a list) and the fitness is invalid because it contains no values.

# Defining a population of individuals

We can now register a population in the toolbox to fill with our individuals

In [ ]:
toolbox1.register("population", tools.initRepeat, list, toolbox1.individual)

And then use it to create an initial population.

In [ ]:
pop = toolbox1.population(n=20)

Let's look at the first individual

In [ ]:
print( pop[0] )

# Evaluation and fitness

The evaluation is the part of the algorithm that evaluates fitness. For some problems evaluation can be done in many different ways, with some being better than others. But this can be a bit of an art and takes some thinking and some experience. In DEAP:
- It is the only part of the library that you must always write yourself.
- A typical evaluation function takes one individual as argument and return its fitness as a tuple.
- A fitness is a list of floating point values and has a property valid to know if this individual shall be re-evaluated
- The fitness is set by setting the values to the associated tuple.

For example, the following evaluates the previously created individual ind1 and assign its fitness to the corresponding values.

In [ ]:
def evaluate(individual):
    # Do some computing on the individual
    a = sum(individual)
    return a,

We will usually return just one fitness measure, but we could return more than one.


In [ ]:
def evaluate(individual):
    # Do some computing on the individual
    a = (1. / sum(individual))
    b = len(individual)
    return a, b

When returning a single fitness measure, the evaluation function must still return a tuple because single-measure fitness is treated as a special case of multi-objective fitness.

To evaluate an individual you can now just call your evaluate function. You pass the full individual, and get back to fitness tuple, which you assign to the individuals fitness value.

In [ ]:
ind1 = pop[0]
ind1.fitness.values = evaluate(ind1)

In [ ]:
print(ind1.fitness.valid)

In [ ]:
print(ind1.fitness)

# Selection

Once you have evaluated the fitness of all individuals in the population, you can select the individuals that have a chance to reproduct into the next generation.

- Selection is made among a population by the selection operators that are available in the deap.operators module.
- The selection operator usually takes as first argument an iterable container of individuals and the number of individuals to select. It returns a list containing the references to the selected individuals.

First we must evaluate the fitness of everyone. We only want to evaluate fitness for individuals who have changed. However, no-one's fitness will be known in the initial population, so we must first evaluate all individuals.

In [ ]:
fitnesses = list(map(evaluate, pop))

This gives us a list of fitness values corresponding to each individual in the population. Next we assign the fitness to each individual. To do this we use the Python zip command to line up the individuals with their corresponding fitness.

In [ ]:
for ind, fit in zip(pop, fitnesses):
    ind.fitness.values = fit

In the following code, we use a naive approach of selecting the top 2 individuals in the population.

In [ ]:
selected = tools.selBest(pop, 2)

Let's see if individual at position 0 in the population managed to be selected

In [ ]:
pop[0] in selected

In [ ]:
print( selected )

Thus, we end up with a list of individuals in 'selected' that may be able to reproduce into the next generation.

# Mutation

- There are a variety of mutation operators in the deap.tools module.
- Each mutation has its own characteristics and may be applied to different type of individual.
- Be sure to read the documentation of the selected operator in order to avoid undesirable behaviour.

To apply a mutation (here a gaussian mutation) on the first individual, simply apply the desired function.

In [ ]:
pop[0]

In [ ]:
tools.mutGaussian(pop[0], mu=0.0, sigma=0.2, indpb=0.2)

Mu and sigma are the mean and standard deviation for the Guassian curve, and indpb is the independent probability of mutation per gene.

You also need to delete the fitness.value for the individual, because it has changed and needs to be re-evaluated the next time we need to its fitness.

In [ ]:
del pop[0].fitness.values

Mutation operators in DEAP are destructive and mutate the original individual in-place. Thus, if you want to keep the original parent pre-mutation, you must make a copy in advance. You will need to do this, because we often want to select individuals 'with replacement', so that the same individual can be selected multiple times. You can clone an individual using the clone function in the toolbox.

In [ ]:
mutant = toolbox1.clone(pop[0])
tools.mutGaussian(mutant, mu=0.0, sigma=0.2, indpb=0.2)
del mutant.fitness.values

# Crossover

- There are a variety of crossover operators in the deap.tools module.
- Each operator has its own characteristics and may be applied to different type of individuals.
- Be careful to read the documentation of the selected operator in order to avoid undesirable behaviour.

As with mutaton, crossover is destructive and thus you often need to make copies of individuals first. Lets apply a crossover operation to produce the two children that are cloned beforehand. Here is an example of cloning the first two individuals in the population and then crossing them over with each other.

In [ ]:
child1, child2 = [ toolbox1.clone(ind) for ind in (pop[0], pop[1]) ]
tools.cxOnePoint(child1, child2)
del child1.fitness.values
del child2.fitness.values

As with mutation, because you have made a change to the individuals, you need to delete their fitness.values so that they will be re-evaluated next time.

Note that the crossover function gives us two individuals, because we sliced two togehter.

In [ ]:
print(pop[0])
print(pop[1])

In [ ]:
print(child1)
print(child2)

Typically you want to implement a probability of crossover.

In [ ]:
cxProb = 0.6

if random.random() < cxProb:
    child1, child2 = [ toolbox1.clone(ind) for ind in (pop[0], pop[1]) ]
    tools.cxOnePoint(child1, child2)
    del child1.fitness.values
    del child2.fitness.values

# Using operators with the toolbox

So far we have only used the toolbox to represent the individual. But the toolbox is also intended to contain all the evolutionary tools, from the object initializers to the evaluation operator. It allows easy configuration of each algorithms.
- The toolbox has basically two methods, register() and unregister(), that are used to add or remove tools from the toolbox.
- The usual names for the evolutionary tools are mate(), mutate(), evaluate() and select(), however, any name can be registered as long as it is unique. Here is how they are registered in the toolbox.
- The main reason to register with the toolbox is to set up an operator with pre-defined parameters.

Remember that we have already defined our toolbox when we defined the individual like this

>from deap import base  
>from deap import tools  
>  
>toolbox1 = base.Toolbox()  
>
>toolbox1.register("attr_float", random.random)  
>toolbox1.register("individual", tools.initRepeat, creator.Individual, toolbox1.attr_float, n=IND_SIZE)

You can register your operators with your toolbox like this

In [ ]:
def evaluateInd(individual):
    # Your evaluation code here
    return result,

toolbox1.register("mate", tools.cxOnePoint)
toolbox1.register("mutate", tools.mutGaussian, mu=0, sigma=1, indpb=0.2)
toolbox1.register("select", tools.selTournament, tournsize=3)
toolbox1.register("evaluate", evaluateInd)

Now you can call, for example, mutate by calling the toolbox, and it will use the default parameters that you have registered (unless you override them).

In [ ]:
mutant = toolbox1.clone(pop[0])
toolbox1.mutate(mutant)
del mutant.fitness.values

# Tool Decorating

A decorator is a wrapper that is called in place of a function. It is asked to make some initialization and termination work before and after the actual function is called. For example, if values need to be constrained, you can apply a decorator to the mutation to keep any individual from going out-of-bounds.

The following defines a decorator that checks if any attribute of the individual is out-of-bounds and clips it if it is the case. The decorator is defined using three functions in order to receive the min and max arguments. Whenever the mutation or crossover is called, bounds will be check on the resulting individuals.

In [ ]:
def checkBounds(min, max):
    def decorator(func):
        def wrapper(*args, **kargs):
            individuals = func(*args, **kargs)
            for indv in individuals:
                for i in range(len(indv)):
                    if indv[i] > max:
                        indv[i] = max
                    elif indv[i] < min:
                        indv[i] = min
            return individuals
        return wrapper
    return decorator

toolbox1.register("mate_example", tools.cxBlend, alpha=0.2)
toolbox1.register("mutate_example", tools.mutGaussian, mu=0, sigma=2)

MIN = 0; MAX = 10

toolbox1.decorate("mate_example", checkBounds(MIN, MAX))
toolbox1.decorate("mutate_example", checkBounds(MIN, MAX))

This will work on crossover and mutation because both return a tuple of individuals. The mutation is often considered to return a single individual but again like for the evaluation, the single individual case is a special case of the multiple individual case.